# NO MACHINE LEARNING LIBARY USED
# sklean ONLY USED for language data into Vector transformation
# preprocess dataset

In [2]:
import pandas as pd
import csv
import codecs
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from sklearn.feature_extraction.text import CountVectorizer#ONLY USED FOR TRANSFORM FROM LANGUAGE TO VECTOR
from numpy import linalg as LA
import math

In [3]:
df = pd.read_csv('./Tab.delimited.Cleaned.dataset.WITH.variable.labels.csv', sep='\t',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df3 = pd.read_csv('./ML3AllSites.csv', sep=',',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
class NeuralNetwork:
    def __init__(self, n_h, n_iterate=10, learning_rate=1):
        self.n_x = None  # size of the input layer
        self.n_h = n_h  # size of the hidden layer
        self.n_y = None # size of the output layer
        self.W1 = None
        self.W2 = None
        self.b1 = None
        self.b2 = None
        self.A1 = None
        self.A2 = None  # sigmoid output of the second activation
        self.learning_rate = learning_rate
        self.n_iterate = n_iterate

    def initialize_parameters(self):
        self.W1 = np.random.randn(self.n_h, self.n_x) * 0.01
        self.b1 = np.zeros((self.n_h, 1))
        self.W2 = np.random.randn(self.n_y, self.n_h) * 0.01
        self.b2 = np.zeros((self.n_y, 1))

    def relu(self, z):
        return z * (z > 0)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_cost(self, Y):
        #cost = np.linalg.norm(self.A2 - Y)
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean()
        return np.squeeze(cost) 

    def forward_propagation(self, X):
        self.A1 = self.relu(self.W1 @ X + self.b1)
        self.A2 = self.sigmoid(self.W2 @ self.A1 + self.b2)

    def backward_propagation(self, X, Y):
        m = X.shape[1]

        dZ2 = self.A2 - Y
        dW2 = dZ2 @ self.A1.T / m
        db2 = np.sum(dZ2, axis=1, keepdims=True) / m
        dZ1 = self.W2.T @ dZ2 * (self.A1 > 0)
        dW1 = dZ1 @ X.T / m
        db1 = np.sum(dZ1, axis=1, keepdims=True) / m

        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def fit(self, X, Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
        self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 500 == 0:
                cost = self.compute_cost(Y)
                self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))

    def predict(self, X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)
        A2 = self.sigmoid(self.W2 @ A1 + self.b2)
        return A2.T

# Need to perform kmeans clustering for each attribute

In [6]:
class KMeans:
    def __init__(self, n_clusters=64):
        self.n_clusters = n_clusters  # number of clusters
        self.centers = None  # to record the centers
        self.labels = None
        self.Y = None

    def random_center(self,Y): #only for 1d dataset
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
            
            
    def random_center2(self,Y):
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y[0]))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
#         for j in range(0):
#             Y_j_min = self.Y[:,j].min()
#             Y_j_max = self.Y[:,j].max()
#             self.centers[:,j] = Y_j_min + (Y_j_max - Y_j_min) * self.centers[:,j]
            
            
            
    def dist(self, point1, point2): #old one
        return 2*(point1[0]-point2[0])**2 + 4*(point1[1]-point2[1])**2 + 3*(point1[2]-point2[2])**2

    def dist2(self,point1,point2):
        return LA.norm(point1-point2)**2
    
    def fit(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    def fit2(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center2(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    
    def transform(self, Y):
        out = np.zeros(Y.shape)
        for i in range(self.n_clusters):
            out[self.labels==i] = self.centers[i]
        return out
    

# oneHOTENCODING

In [7]:
def oneHotEncoding(Y, kmeans):
    out = np.zeros((len(Y), kmeans.n_clusters))
    for i in range(len(Y)):
        out[i, Y[i]] = 1
    return out

# test clustering numbered data:
    step1 : get single column
    step2 : sort in increasing order
    step3 : based on distribution, cut data into 4 section with equally number of candidate
    step4 : if data is null, set isnon to be true for that column
    step5 : append new datacol to input_arr

In [54]:
def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

def isfloat(value):
  try:
    isfloat(value)
    return True
  except ValueError:
    return False

def getdistriarr(df,arr_str):
    arr = df[arr_str].values
    di = []
    di_og = []  #RETURN OG SINCE di will be sort
    for i in range(len(arr)):
        if isint(arr[i]) == True:
            di.append(int(arr[i]))
            di_og.append(int(arr[i]))
        else:
            di.append(-1)
            di_og.append(-1)
    di.sort()
    ans_arr = []
    leng = len(di)
    ans_arr.append(di[int(leng/3)])
#     ans_arr.append(di[int(leng/2)])
    ans_arr.append(di[int(leng*0.66)])
    ans_arr.append(di[int(leng-1)])
    return di_og,ans_arr

def oneHotEncoding_return_arr(att_arr,class_arr): #one 
    #add one more column for each row since last col used as flag
    choice = len(class_arr)
    out = np.zeros((len(att_arr),len(class_arr)+1))
    for i in range(len(att_arr)):
        loc = 0
        #first jude if data is null
        if att_arr[i] == -1:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            for j in range(len(class_arr)):
                if att_arr[i] < class_arr[j]:
                    out[i][j] = 1
                    break
        
    return out


def GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,attr_name,og_dic):
    att_arr,class_arr = getdistriarr(df,attr_name)
    dic_ans = create_dic_for_classification(class_arr)
    append_arr_of_dic_to_overall(og_dic,dic_ans,attr_name)

    return oneHotEncoding_return_arr(att_arr,class_arr)

def create_dic_for_classification(class_arr):
    cur_dic = {}
    total_len = len(class_arr)
    for i in range(len(class_arr)):
        cur_dic[class_arr[i]] = i
    return cur_dic

def append_arr_of_dic_to_overall(og,dic,att_name):
    og[att_name] = dic
    
    
def append_arr(old,new):
    #both old and new has the same num of row
    #create a new nparray
    
    newdim = len(old[0])+len(new[0])
    ans = np.zeros((len(old),newdim))
    print(ans.shape)
    for i in range(len(old)):
        ans[i] = np.append(old[i],new[i])
    return ans

    
    





# STEP TWO CONVERT NLP DATASET INTO VECTOR:
    STEP1: vectorize dataset
    STEP2: apply kmeans clustering to language data
    STEP3: ONEHOT ENCODING
    STEP4: APPEND CLASSIFICATION DATASET INTO old CLASSFICAITION ARR

In [9]:
def NLPDATAPRO(allsentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(allsentences)
    ans = X.toarray()
    return ans

def createtextarr_ONEHOT_ENCODING(attri_col): #ONEHOT_ENCODING
    df2 = df[attri_col]
    arr = df2.values
    for i in range(len(arr)):
        di.append(arr[i])
    return arr

# STEP THREE CONVERT TEXT_DATASET WITH FIXED classification into vector

In [10]:
def processnull(arr):
    arrnew = []
    for i in range(len(arr)):
        if type(arr[i]) == float and math.isnan(arr[i]) :
            arrnew.append('Nan')
        else:
            arrnew.append(arr[i])
    return arrnew

def returnuniqiearr(og):
    ar = np.asarray(og)
    return np.unique(ar)

# TODO: create a function to append text to old vecto(remember to append flag)

# TODO: TEST NN TONIGHT?

In [11]:
def create_onehoted_text_fixed_classification(attribu_name,df,km_num):
    df_cur = df[attribu_name]
    attribute_arr = df_cur.values
    arr_no_null = processnull(attribute_arr) #create new arr with format fit by NLP(CountVectorizer)
    NLP_ARR = NLPDATAPRO(arr_no_null)
    #BEFORE ONEHOT_ENCODING
    #APPLY KMEANS CLUSTERING SINCE TOO MANY DIFFERENT CLASSIFICATION
    KM_TMP = KMeans(km_num)
    KM_TMP.fit2(NLP_ARR)
    labels_arr = KM_TMP.labels
    return labels_arr
    
def onehot_enco(labelarr,df,att_name):
    og_text_arr = df[att_name].values
    #create dic to remeber loc for each value in the uniqe arr
    uni_arr = np.unique(labelarr)
    loc_dic = {}
    for i in range(len(uni_arr)):
        loc_dic[uni_arr[i]] = i
    #based on the lenth of dic,create onehot_encod
    choice = len(loc_dic)
    out = np.zeros((len(labelarr),choice+1))
    for i in range(len(labelarr)):
        #first junde if data is null
        if type(og_text_arr[i])!=str and math.isnan(og_text_arr[i]) == True:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            cur_loc = loc_dic.get(labelarr[i])
            out[i][cur_loc] = 1
            
    return out,loc_dic

def GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,attr_name,og_dic,km_num):
    labels_arr = create_onehoted_text_fixed_classification(attr_name,df,km_num)
    out_arr,loc_dic = onehot_enco(labels_arr,df,attr_name)
    #append dic to global array of dic
    append_arr_of_dic_to_overall(og_dic,loc_dic,attr_name)
    return out_arr

# GENERAL WORK:
    1. MAINTAIN AN ARRAY OF DICTIONARY S.T EVERY DIC
       CONTAINS MAPPING FROM CLASSIFICAITON TO ACTUAL INDEX AFTER KMEANS
    
    2. AFTER TRAINING, WE WILL UTILIZED THE ARR_DIC TO RECOVER CANDIDATE ANSER
    
    3. TODO: REWRITE COST FUNCTION OF NERUAL NETWORK
    

# FINDING:
    AFTER combine several attributes column, num of candidate with dinct
    ans grows extremly fast.
    Need to apply Kmeans clustering again to the array of candidate before
    threw into Neural network

# CREATE A SCRIPT TO APPEND ASSIGNED ATTRIBUTE TO INPUT COLUMN

(API)INSTRUCTION:<br>
USER WHO WANTS TO CREATE PREPROCESSED DATASET ONLY NEED TO CREATE
ARRAY[i]<br>
       ARRAY[i][0] = 'name of attribute'<br>
       ARRAY[i][1] = 1 : it is a digit column<br>
       ARRAY[i][1] = 0 : it is a NLP column



In [12]:
def semi_auto_append_attri(arr_of_attribute,df,og_dic):
    for i in range(len(arr_of_attribute)):
        name = arr_of_attribute[i][0]
        isdigit = arr_of_attribute[i][1]
        if isdigit == 1:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,name,og_dic)
        else:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,name,og_dic,20)
        #append old with new
        if i == 0:
            old = attr_arr
        else:
            old = append_arr(old,attr_arr)
#             print(old.shape)
    return old
        

# Apply Kmeans Clustering to the overal column

In [13]:
def cluster_whole_attribute_get_Y_for_Neural_NetWork(X,num_cluster):
    km = KMeans(num_cluster)
    km.fit2(X)
    
    #APPLY ONEHOT_ENCODING TO NEURAL_NETWORKAGAIN
    
    return oneHotEncoding(km.labels,km),km

# APPLY NEURAL NETWORK TO SOLVE THIS QUESTION:

REMEBER TO CONSIDER FLAG FOR EACH COLUMN

# QUESTION: SHOULD I APPLY ONE HOT ENCODING TO EVERY ATTRIBUT?

In [14]:
def process_predicted_data(pred,kmeans):
    ans = []
    for i in range(len(pred)):
        ans.append(kmeans.centers[np.argmax(pred[i])])
    newarr = np.asarray(ans)
    return newarr

def find_classification_based_on_distance(pred,dic):
    #create array with all dict keys based on insertion order
    dicarr = list(dic.keys())
    #isolate target awated_array used to classification
    for i in range(len(pred)):
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
#             print('cur dic: ',cur_dict_key)
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_lengh = len(cur_dict_arr)+1
            cur_des += cur_dict_lengh  #update current cutting position
            #restore dimension of onehotencoding for current dictionary
            one_hot_cur_dic = np.zeros((cur_dict_lengh,cur_dict_lengh))
#             print('before any processing cur loc is ', cur_loc, ' cur des is : ', cur_des)
            for q in range(0,cur_dict_lengh-1):
                one_hot_cur_dic[q][q] = 1
                one_hot_cur_dic[q][-1] = 1
            #isolate target awaited_classification array
            cur_isolated_arr = pred[i][cur_loc:cur_des]
            #compute classification with smallest distance
            arr_store_distance= []
            for h in range (len(one_hot_cur_dic)):
                arr_store_distance.append(np.linalg.norm(cur_isolated_arr-one_hot_cur_dic[h]))
            #transfer dic to arr s.t unilized argmim
            dis_np_array = np.asarray(arr_store_distance)
            index = np.argmin(dis_np_array)
#             if cur_dict_key == 'big5_10':
#                 print(index)
#                 print('cur onehot_array is ', one_hot_cur_dic[3] )
#                 print('cur loc is ', cur_loc, ' cur des is : ', cur_des)
            # change value in output array
            for k in range (cur_dict_lengh):
                pred[i][cur_loc+k] = one_hot_cur_dic[index][k]
                
            cur_loc += cur_dict_lengh #update cur_loc not used in curretn loop

def create_reverse_dictionary(dic):
    reverse = {}
    dic_arr = list(dic.keys())
    for i in range(len(dic_arr)):
        cur_reverse_dict = {}
        cur_dic_key = dic_arr[i]
        cur_key_arr = list(dic.get(cur_dic_key).keys())
        cur_index_arr = list(dic.get(cur_dic_key).values())
        #reverse key_value pair in originaly dictionary
        for j in range(len(cur_key_arr)):
            cur_reverse_dict[cur_index_arr[j]] = cur_key_arr[j]
        #append current dic to ans dic
        reverse[cur_dic_key] = cur_reverse_dict
    return reverse

In [19]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr.append(['age',1])
arr.append(['mood_01',1]) #Today I generally feel
arr.append(['major',0])
arr.append(['big5_01',1]) #I see myself as: Extraverted, enthusiastic.
arr.append(['big5_02',1]) #I see myself as: Critical, quarrelsome.
arr.append(['big5_03',1]) #I see myself as: Dependable, self-disciplined.
arr.append(['big5_04',1]) #I see myself as: Anxious, easily upset.
arr.append(['big5_05',1]) #I see myself as: Open to new experiences, complex.
arr.append(['big5_06',1]) #I see myself as: Reserved, quiet.
arr.append(['big5_07',1]) #I see myself as: Sympathetic, warm.
arr.append(['big5_08',1]) #I see myself as: Disorganized, careless.
arr.append(['big5_09',1]) #I see myself as: Calm, emotionally stable.
arr.append(['big5_10',1]) #I see myself as: Conventional, uncreative.

In [20]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

In [21]:
# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)

(2998, 8)


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
(2998, 18)
(2998, 22)
(2998, 26)
(2998, 30)
(2998, 34)
(2998, 38)
(2998, 42)
(2998, 46)
(2998, 50)
(2998, 54)
(2998, 58)


In [22]:
#slice nparr
x_pred = X_train[2000:]
x_real_train = X_train[:2000]

In [23]:
np.unique(x_real_train,axis = 0).shape

(1555, 58)

In [24]:
nn = NeuralNetwork(50,2000,0.1) #size of input layer

In [25]:
nn.fit(x_real_train,x_real_train)

Cost after iteration 0: 0.693188
Cost after iteration 500: 0.285494
Cost after iteration 1000: 0.285439
Cost after iteration 1500: 0.285407


In [160]:
ans = nn.predict(x_pred)

In [161]:
find_classification_based_on_distance(ans,og_pred)

# TODO : CREATE FUNCTION TO RECOVER CLASSIFICATION DATASET INTO HUMAN-Fridendly DATASET

In [163]:
def recover_answer(predict_arr,dic):
    reverse_dic = create_reverse_dictionary(dic)
    #init an dic as final anser array
    ans_arr = []
    #create array with all dict keys based on insertion order
    dic_arr = list(dic.keys())
    #recover predicted candidate sequentilaay
    for i in range(len(predict_arr)):
        tmp_ans_array = {} # cur dict to store anser for this candidate
        cur_loc = 0
        cur_des = 0
        #loop through dictionary array to recover each attribute column
        for j in range(len(dic_arr)):
            cur_dict_key = dic_arr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_reverse_dic_arr = reverse_dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length #update current cutting position
            #restore
            tmp_cutting_arr = predict_arr[i][cur_loc:cur_des]
            tag = tmp_cutting_arr[-1]
            if tag == 0:
                tmp_ans_array[cur_dict_key] = 'NA'
            else:
                #find the position where tag is 1
                loc = 0
                for pos in range(len(tmp_cutting_arr)):
                    if tmp_cutting_arr[pos] == 1:
                        loc = pos
                        break
                #now based on location, create cover anser
                #case one loc is 0
                if loc == 0:
                    ans ="<" + str(cur_reverse_dic_arr.get(loc))
                    tmp_ans_array[cur_dict_key] = ans
                else:
                    last_loc = loc-1
                    ans=""
                    ans+=str(cur_reverse_dic_arr.get(last_loc))+" < "
                    ans+=str(cur_reverse_dic_arr.get(loc))
                    tmp_ans_array[cur_dict_key] = ans
#                 ans =cur_reverse_dic_arr.get(loc)
#                 tmp_ans_array[cur_dict_key] = ans
        
            cur_loc+=cur_dict_length
        #append current tmp candidate array to finaly answer
        ans_arr.append(tmp_ans_array)
    
    return ans_arr
                    

# OUT OF 198 "untrained" ATTRIBUTE BY NEURAL NETWORK", THERE ARE 34 Candidate be classied into right section without error

In [612]:
count = 0
total = 0
for index in range(len(x_pred)):
    for i in range(len(x_pred[index])):
        if x_pred[index][i] != ans_after[index][i]:
            count+=1
    if count<=5:
        total+=1
    count = 0
print('total',total)

total 34


In [39]:
count = 0
total = 0
for index in range(len(x_pred)):
    for i in range(len(x_pred[index])):
        if x_pred[index][i] != ans[index][i]:
            count+=1
    if count<=5:
        total+=1
    count = 0
print('total',total)

total 87


# START WORKING ON: SENTIMENT ANALYSIS FOR NLP COLUMN

# THOUGHTS: Right now, i am attaching a flag for every after_one_hot_encoding_attribute_vector<br>
Means: the model will likely(100%) predict something that is not valid
TODO: rewrite dataset representation, create a single flag array, when computing the cost function
we use sqart(y-x)*flag

Problem: if flag == 0, cost = 0? WTF???

In [115]:
og_pred

{'age': {18: 0, 19: 1, 149: 2},
 'mood_01': {2: 0, 3: 1, 7: 2},
 'major': {0: 0, 2: 1, 4: 2, 6: 3, 7: 4, 9: 5, 12: 6, 13: 7, 17: 8},
 'big5_01': {3: 0, 6: 1, 7: 2},
 'big5_02': {2: 0, 5: 1, 7: 2},
 'big5_03': {5: 0, 6: 1, 7: 2},
 'big5_04': {2: 0, 5: 1, 7: 2},
 'big5_05': {5: 0, 6: 1, 7: 2},
 'big5_06': {2: 0, 5: 1, 7: 2},
 'big5_07': {5: 0, 6: 1, 7: 2},
 'big5_08': {1: 0, 3: 1, 7: 2},
 'big5_09': {4: 0, 6: 1, 7: 2},
 'big5_10': {2: 0, 3: 1, 7: 2}}

In [121]:
x_pred[0][0:4]

array([0., 0., 1., 1.])

# Thoughts: two column in manylab 3 contains NLP COLUMN, HIGHPOWER & LOWPOWER, I could analysis these two column to tag sentiment for each candidate, to evaalute whether they like highpower and lowpower or not.

# SENTIMENT CLASS: ENJOR > NEURTAL > NEGATIVE

In [27]:
X_train.shape

(2998, 58)

# CREATE FUNCTION FOR CROSS VALIDATION
Assess the accuracy by count how many

In [75]:
def cross_validation(arr_all_candidate,num_fold,og_pred):
    cost = 0
    #create num_fold equal size fold from input nparray
    list_of_fold = np.array_split(arr_all_candidate,num_fold)
    
    #need to append training dataset together, while leave testing set alone
    for i in range(num_fold):
        x_pred = list_of_fold[i]
        
        for j in range(num_fold):
            ans = 0
            if j != i:
                if ans == 0:
                    ans = np.asarray(list_of_fold[j])
                else:
                    ans = cross_valid_append_arr(ans,list_of_fold[j])
        #finished rebuilding training dataset
        #start training neural network
        nn = NeuralNetwork(50,2000,0.1) #size of input layer
        nn.fit(ans,ans)
        predict_ans = nn.predict(x_pred)
        find_classification_based_on_distance(predict_ans,og_pred)
        #coun
        tmpcost = cost_function(predict_ans,x_pred)
        cost+=tmpcost
        print('#### current cutting position is : ', i, 'current cost is: ',tmpcost)
    print('final avg cost is : ', cost/num_fold)
    return cost/num_fold



def cost_function(x,y):
    #compute the average num of wrongly predicted column as cost function
    total_num = len(x)
    count = 0
    
    for i in range(len(x)):
        tmpcount = 0
        for j in range(len(x[i])):
            if x[i][j]!=y[i][j]:
                tmpcount+=1
        count+=tmpcount
    avg_count = count/total_num
    return avg_count
def cross_valid_append_arr(old,new):
    ans = np.append([old],[new],axis = 0)
    return ans

# TEST CROSS VALIDATION

In [76]:
cross_validation(X_train,10,og_pred)

Cost after iteration 0: 0.693177
Cost after iteration 500: 0.436549
Cost after iteration 1000: 0.436534
Cost after iteration 1500: 0.436527
#### current cutting position is :  0 current cost is:  14.486666666666666
Cost after iteration 0: 0.693171
Cost after iteration 500: 0.436549
Cost after iteration 1000: 0.436534
Cost after iteration 1500: 0.436527
#### current cutting position is :  1 current cost is:  16.83
Cost after iteration 0: 0.693087
Cost after iteration 500: 0.436549
Cost after iteration 1000: 0.436534
Cost after iteration 1500: 0.436527
#### current cutting position is :  2 current cost is:  19.99
Cost after iteration 0: 0.693160
Cost after iteration 500: 0.436549
Cost after iteration 1000: 0.436534
Cost after iteration 1500: 0.436527
#### current cutting position is :  3 current cost is:  14.906666666666666
Cost after iteration 0: 0.693044
Cost after iteration 500: 0.436549
Cost after iteration 1000: 0.436534
Cost after iteration 1500: 0.436527
#### current cutting posit

AttributeError: 'int' object has no attribute 'T'